In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, Random, Distributions, LinearAlgebra
import Plots: Plot
import JuliaProbo: AbstractObject, AbstractWorld, AbstractAgent, AbstractEstimator, Landmark, Map, KalmanFilter, EstimatorAgent, RealRobot, RealCamera, World, Goal, Puddle, PuddleWorld, puddle_depth, observations, push!, draw, decision, state_transition, motion_update, observation_update, inside
gr();

In [ ]:
mutable struct PuddleIgnoreAgent <: AbstractAgent
    v_::Float64
    ω_::Float64
    dt::Float64
    prev_v_::Float64
    prev_ω_::Float64
    estimator_::AbstractEstimator
    puddle_coeff::Float64
    puddle_depth_::Float64
    total_reward_::Float64
    in_goal_::Bool
    final_value_::Float64
    goal::Goal
    function PuddleIgnoreAgent(
        v::Float64,
        ω::Float64,
        dt::Float64,
        estimator::AbstractEstimator,
        goal::Goal;
        puddle_coeff=100
    )
        new(v, ω, dt, 0.0, 0.0, estimator, puddle_coeff, 0.0, 0.0, false, 0.0, goal)
    end
end

function reward_per_sec(agent::PuddleIgnoreAgent)
    return -1.0 - agent.puddle_depth_ * agent.puddle_coeff
end

function policy(agent::PuddleIgnoreAgent, goal::Goal)
    cur_bel_pose = agent.estimator_.pose_
    x, y, θ = cur_bel_pose[1], cur_bel_pose[2], cur_bel_pose[3]
    dx, dy = goal.x - x, goal.y - y
    direction = convert(Int64, round((atan(dy, dx) - θ) * 180 / pi))
    while direction > 180
        direction -= 360
    end
    while direction <= -180
        direction += 360
    end
    v, ω = 0.0, 0.0
    if direction > 10
        v, ω = 0.0, 2.0
    elseif direction < -10
        v, ω = 0.0, -2.0
    else
        v, ω = 1.0, 0.0
    end
    v, ω
end

function decision(agent::PuddleIgnoreAgent, observation::Vector{Vector{Float64}})
    if agent.in_goal_
        return 0.0, 0.0
    end    
    motion_update(agent.estimator_,
        agent.prev_v_,
        agent.prev_ω_,
        agent.dt)
    obseravtion = Vector{Vector{Float64}}(undef, 0)
    observation_update(agent.estimator_, observation)
    agent.total_reward_ += agent.dt * reward_per_sec(agent)
    
    v, ω = policy(agent, agent.goal)
    agent.prev_v_, agent.prev_ω_ = v, ω
    return v, ω
end

function draw(agent::PuddleIgnoreAgent, p::Plot{T}) where {T}
    x, y = agent.estimator_.pose_[1], agent.estimator_.pose_[2]
    annota1 = "reward/sec: $(round(reward_per_sec(agent), sigdigits=3))"
    annota2 = "total reward: $(round(agent.total_reward_ + agent.final_value_, sigdigits=3))"
    p = annotate!(x+1.0, y-0.5, text(annota1, 10))
    p = annotate!(x+1.0, y-1.0, text(annota2, 10))
end

function update_status(world::PuddleWorld)
    for iter in 1:lastindex(world.robots_)
        robot = world.robots_[iter]
        # use the ground truth position
        robot.agent_.puddle_depth_ = puddle_depth(world, robot.pose_)
        for g in world.goals_
            if inside(g, robot.pose_)
                robot.agent_.in_goal_ = true
                robot.agent_.final_value_ = g.value
            end
        end
    end
end

function ch10_puddle_world4()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks =
        [Landmark([-4.0, 2.0], 0), Landmark([2.0, -3.0], 1), Landmark([4.0, 4.0], 2),  Landmark([-4.0, -4.0], 3)]
    envmap = Map()
    push!(envmap, landmarks)
    world = PuddleWorld(xlim, ylim)
    push!(world, envmap)
    # goal
    goal = Goal(-3.0, -3.0)
    push!(world, goal)
    # robot side
    initial_pose = [2.0, 2.0, 0.0]
    estimator = KalmanFilter(envmap, initial_pose)
    agent = PuddleIgnoreAgent(0.2, 10.0 * pi / 180, dt, estimator, goal)
    robot = RealRobot(initial_pose, agent, RealCamera(landmarks, distance_bias_rate_stddev = 0.0, direction_bias_stddev = 0.0); color = "red")
    push!(world, robot)
    # puddles
    push!(world, Puddle([-2.0, 0.0], [0.0, 2.0], 0.1))
    push!(world, Puddle([-0.5, -2.0], [2.5, 1.0], 0.1))
    
    anim = @animate for i = 1:300
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        # t
        update_status(world)
        z = observations(robot.sensor_, robot.pose_; noise = false, bias = false)
        p = draw(world, annota)
        
        # t+1
        v, ω = decision(agent, z)
        state_transition(robot, v, ω, dt; move_noise = true, vel_bias_noise = true)
    end
    gif(anim, "images/ch10_puddle_world4.gif", fps = 10)
end
ch10_puddle_world4()

<img src="images/ch10_puddle_world4.gif">